In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib
from talib import abstract
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
import xgboost as xgb
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.optimizers import Adam

In [2]:
# Load data
file_path = 'TXF.pkl'
df = pd.read_pickle(file_path)

# Ensure index is in datetime format
df.index = pd.to_datetime(df.index)

# Drop unnecessary columns and rename others
df.drop(['簡稱', '期貨名稱'], axis=1, inplace=True)
df.columns = ['open', 'high', 'low', 'close', 'returns', 'volume', 'open_int', 'basis', 'tot_unsettled']

In [3]:

# Add basic technical indicators
df['sma'] = abstract.MA(df, 20, matype=0)
df['ema'] = abstract.MA(df, 20, matype=1)
df['adx'] = abstract.ADX(df)
df['rsi'] = abstract.RSI(df)
macd_values = abstract.MACD(df)
df = pd.concat([df, macd_values], axis=1)

# Drop NA values
df = df.dropna()

# Select initial features
features = ['open', 'high', 'low', 'volume', 'open_int', 'basis', 'tot_unsettled', 'sma', 'ema', 'adx', 'rsi', 'macd', 'macdsignal', 'macdhist']
X = df[features]

In [4]:
# Add polynomial features and interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_poly = poly.fit_transform(X)

# Shift target variable to predict the next period's close
y = df['close'].shift(-1).dropna()

# Align X with y
X_poly = X_poly[:-1]
X = X.iloc[:-1]

In [27]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_poly)


# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Apply Lasso regression for feature selection
lasso = LassoCV(cv=5, random_state=42, max_iter= 10000, alphas = [0.01, 0.1, 1, 10] ) 
lasso.fit(X_train, y_train)

# Check selected features
lasso_selected_features = lasso.coef_ != 0
selected_features = np.array(poly.get_feature_names_out(features))[lasso_selected_features]
print(selected_features)
# Sort features by the absolute value of their coefficients
sorted_indices = np.argsort(np.abs(lasso.coef_))[::-1]
n_top_features = 8  # Specify the number of top features you want to keep

# Select the top n features based on their coefficient values
top_features_indices = sorted_indices[: n_top_features]
top_selected_features = selected_features[top_features_indices]

print("Top Selected Features by Lasso regression:")
top_selected_features

/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15765401.804149752, tolerance: 5502375.608796689
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17669042.111037917, tolerance: 5481408.17821956
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16660491.57494294, tolerance: 5418669.1447733315
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:617: 

['open' 'high' 'low' 'volume' 'basis' 'tot_unsettled' 'sma' 'ema' 'adx'
 'macd' 'macdsignal' 'macdhist' 'open^2' 'open high' 'open low'
 'open volume' 'open open_int' 'open basis' 'open tot_unsettled'
 'open sma' 'open ema' 'open adx' 'open rsi' 'open macd' 'open macdhist'
 'high^2' 'high low' 'high volume' 'high open_int' 'high basis'
 'high tot_unsettled' 'high sma' 'high ema' 'high adx' 'high rsi'
 'high macd' 'high macdsignal' 'low^2' 'low volume' 'low basis'
 'low tot_unsettled' 'low sma' 'low adx' 'low rsi' 'low macd'
 'low macdhist' 'volume^2' 'volume open_int' 'volume basis'
 'volume tot_unsettled' 'volume ema' 'volume adx' 'volume rsi'
 'volume macd' 'volume macdhist' 'open_int^2' 'open_int basis'
 'open_int tot_unsettled' 'open_int sma' 'open_int adx' 'open_int rsi'
 'open_int macd' 'open_int macdhist' 'basis^2' 'basis tot_unsettled'
 'basis sma' 'basis adx' 'basis rsi' 'basis macdsignal' 'basis macdhist'
 'tot_unsettled^2' 'tot_unsettled sma' 'tot_unsettled adx'
 'tot_unsett

/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.578e+07, tolerance: 6.809e+06
  model = cd_fast.enet_coordinate_descent(


array(['high', 'open low', 'low', 'open macd', 'open', 'volume basis',
       'high^2', 'low adx'], dtype=object)

In [28]:
# Convert X_train and X_test to pandas DataFrames
X_train_df = pd.DataFrame(X_train, columns=poly.get_feature_names_out(features))
X_test_df = pd.DataFrame(X_test, columns=poly.get_feature_names_out(features))

# Use selected features for the final model
X_selected_train = X_train_df[top_selected_features]
X_selected_test = X_test_df[top_selected_features]

# Convert back to NumPy arrays if needed (for compatibility with some models)
X_selected_train = X_selected_train.values
X_selected_test = X_selected_test.values

print("Selected features for training set:")
print(X_selected_train)


Selected features for training set:
[[ 1.60699645e-01  1.62271950e-01  1.56991903e-01 ...  7.26264477e-02
   1.61657150e-01  7.16135877e-01]
 [-1.01120525e-02 -6.08252614e-02 -5.62645808e-02 ... -2.24153266e-01
  -9.79946233e-03  1.19519598e-01]
 [ 1.27575840e+00  1.52062891e+00  1.17551805e+00 ...  1.14930753e+01
   1.59042691e+00  8.69107834e-01]
 ...
 [ 5.88958732e-01  5.81351046e-01  5.40688898e-01 ...  4.50588623e+00
   6.46913022e-01  4.56996465e-01]
 [ 5.80213173e-01  5.96931302e-01  5.48922739e-01 ... -6.55694590e-01
   6.36211565e-01  5.54961660e-01]
 [-6.74774529e-01 -5.89619684e-01 -7.50651846e-01 ... -3.45957642e-01
  -5.57103978e-01  8.68236103e-01]]


## Model Training 

In [36]:
import time
import itertools
from sklearn.ensemble import VotingRegressor

# Set initial best score
best_score = -float('inf')
best_model_name = None
best_model = None
best_params = None

# Define reduced ranges for hyperparameters and feature engineering options
C_range = [0.1, 10]  # Reduced options
epsilon_range = [0.01, 0.1]  # Reduced options
gamma_range = ['scale']  # Reduced options
polynomial_degrees = [2]  # Reduced to one degree

# Initialize time tracking
start_time = time.time()
iteration_times = []

# Total number of iterations for SVR grid search
total_svr_iterations = len(polynomial_degrees) * len(C_range) * len(epsilon_range) * len(gamma_range)

# Start grid search loop
iteration_count = 0
for degree in polynomial_degrees:
    print(f"Testing with Polynomial Degree: {degree}")
    
    # Update polynomial features
    poly = PolynomialFeatures(degree=degree, interaction_only=False, include_bias=False)
    X_poly = poly.fit_transform(X)
    
    # Standardize features
    X_scaled = scaler.fit_transform(X_poly)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # Apply Lasso regression for feature selection
    lasso = LassoCV(cv=3, random_state=42)  # Reduced cross-validation folds
    lasso.fit(X_train, y_train)
    lasso_selected_features = lasso.coef_ != 0
    X_train_sel = X_train[:, lasso_selected_features]
    X_test_sel = X_test[:, lasso_selected_features]
    
    # Loop through reduced SVR hyperparameters
    for C, epsilon, gamma in itertools.product(C_range, epsilon_range, gamma_range):
        iteration_start_time = time.time()  # Track time for each iteration
        iteration_count += 1
        print(f"Testing with C: {C}, Epsilon: {epsilon}, Gamma: {gamma}")
        
        svr = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma=gamma)
        svr.fit(X_train_sel, y_train)
        
        # Evaluate the model
        y_pred = svr.predict(X_test_sel)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        
        print(f"R2 Score: {r2}, MSE: {mse}")
        
        # Check for improvement
        if r2 > best_score:
            best_score = r2
            best_model_name = "SVR"
            best_model = svr
            best_params = {'C': C, 'epsilon': epsilon, 'gamma': gamma, 'degree': degree}
            print(f"New Best Model Found: {best_model_name} with R2 Score: {best_score}")
        
        # Calculate time for this iteration
        iteration_end_time = time.time()
        iteration_duration = iteration_end_time - iteration_start_time
        iteration_times.append(iteration_duration)
        
        # Estimate time remaining
        avg_iteration_time = sum(iteration_times) / len(iteration_times)
        remaining_iterations = total_svr_iterations - iteration_count
        estimated_time_remaining = remaining_iterations * avg_iteration_time
        
        # Display estimated finish time
        print(f"Estimated time remaining: {estimated_time_remaining / 60:.2f} minutes")
    
    # Test XGBoost with reduced grid search
    param_grid = {
        'n_estimators': [100, 200, 300],  # Reduced options
        'learning_rate': [0.01, 0.05, 0.1],  # Reduced options
        'max_depth': [3, 5, 7],  # Reduced options
        'subsample': [0.8, 1.0],  # Reduced options
        'colsample_bytree': [0.8],  # Reduced options
        'gamma': [0, 0.1],  # Reduced options
        'alpha': [0, 0.1],  # Reduced options
        'lambda': [1.0, 1.5, 2.0]  # Reduced options
    }
    
    grid_search = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
                               param_grid=param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=2 )  # Reduced cross-validation folds
    grid_search.fit(X_train_sel, y_train)
    
    # Evaluate the model
    y_pred = grid_search.best_estimator_.predict(X_test_sel)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    print(f"XGBoost R2 Score: {r2}, MSE: {mse}")
    
    # Check for improvement
    if r2 > best_score:
        best_score = r2
        best_model_name = "XGBoost"
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        print(f"New Best Model Found: {best_model_name} with R2 Score: {best_score}")

# Output the best model and its parameters
print(f"Best Model: {best_model_name}")
print(f"Best R2 Score: {best_score}")
print(f"Best Parameters: {best_params}")


Testing with Polynomial Degree: 2


/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.235e+07, tolerance: 6.809e+06
  model = cd_fast.enet_coordinate_descent(


Testing with C: 0.1, Epsilon: 0.01, Gamma: scale
R2 Score: -0.043404707754231886, MSE: 13836362.814814815
New Best Model Found: SVR with R2 Score: -0.043404707754231886
Estimated time remaining: 0.05 minutes
Testing with C: 0.1, Epsilon: 0.1, Gamma: scale
R2 Score: -0.043404707754231886, MSE: 13836362.814814815
Estimated time remaining: 0.03 minutes
Testing with C: 10, Epsilon: 0.01, Gamma: scale
R2 Score: 0.6715262095571667, MSE: 4355819.468657232
New Best Model Found: SVR with R2 Score: 0.6715262095571667
Estimated time remaining: 0.02 minutes
Testing with C: 10, Epsilon: 0.1, Gamma: scale
R2 Score: 0.6715313383193396, MSE: 4355751.457257926
New Best Model Found: SVR with R2 Score: 0.6715313383193396
Estimated time remaining: 0.00 minutes
Fitting 3 folds for each of 648 candidates, totalling 1944 fits
[CV] END alpha=0, colsample_bytree=0.8, gamma=0, lambda=1.0, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END alpha=0, colsample_bytree=0.8,

In [49]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score


# Best model from the grid search
final_model = xgb.XGBRegressor(objective='reg:squarederror',
                               colsample_bytree=0.8,
                               learning_rate=0.05,
                               max_depth=5,
                               n_estimators=200,
                               subsample=1.0,
                               random_state=42, 
                               alpha= 0, 
                               gamma = 0) 

# Define the TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Perform cross-validation using TimeSeriesSplit
cv_scores = cross_val_score(final_model, X_scaled, y, cv=tscv, scoring='r2', n_jobs=-1)

# Output the cross-validation scores
print("Cross-Validation R2 Scores: ", cv_scores)
print("Mean Cross-Validation R2 Score: ", np.mean(cv_scores))
print("Standard Deviation of Cross-Validation R2 Scores: ", np.std(cv_scores))


Cross-Validation R2 Scores:  [ 0.97848249  0.97458264  0.9667295   0.41054318 -2.3664962 ]
Mean Cross-Validation R2 Score:  0.1927683236661742
Standard Deviation of Cross-Validation R2 Scores:  1.2980644869855076


In [47]:
# Adjusted LSTM/GRU model with more layers and increased training epochs

from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.optimizers import Adam

def build_rnn_model(model_type, input_shape):
    model = Sequential()
    if model_type == "LSTM":
        model.add(LSTM(units=100, return_sequences=True, input_shape=input_shape))
        model.add(Dropout(0.3))
        model.add(LSTM(units=100, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(LSTM(units=50))
    elif model_type == "GRU":
        model.add(GRU(units=100, return_sequences=True, input_shape=input_shape))
        model.add(Dropout(0.3))
        model.add(GRU(units=100, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(GRU(units=50))
    
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

# Use a longer training process
model = build_rnn_model("LSTM", input_shape=(X_rnn_train.shape[1], X_rnn_train.shape[2]))
history = model.fit(X_rnn_train, y_train, epochs=100, batch_size=32, validation_data=(X_rnn_test, y_test), verbose=2)

# Evaluate the model
y_rnn_pred = model.predict(X_rnn_test)
mse = mean_squared_error(y_test, y_rnn_pred)
r2 = r2_score(y_test, y_rnn_pred)

print(f"LSTM Model - Mean Squared Error: {mse}")
print(f"LSTM Model - R2 Score: {r2}")


Epoch 1/100


/opt/homebrew/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


161/161 - 3s - 22ms/step - loss: 94870640.0000 - val_loss: 95228504.0000
Epoch 2/100
161/161 - 1s - 3ms/step - loss: 94633384.0000 - val_loss: 95083336.0000
Epoch 3/100
161/161 - 1s - 3ms/step - loss: 94499928.0000 - val_loss: 94956192.0000
Epoch 4/100
161/161 - 1s - 3ms/step - loss: 94376304.0000 - val_loss: 94834664.0000
Epoch 5/100
161/161 - 1s - 3ms/step - loss: 94256584.0000 - val_loss: 94715928.0000
Epoch 6/100
161/161 - 0s - 3ms/step - loss: 94138992.0000 - val_loss: 94598744.0000
Epoch 7/100
161/161 - 1s - 3ms/step - loss: 94022776.0000 - val_loss: 94482520.0000
Epoch 8/100
161/161 - 1s - 3ms/step - loss: 93907440.0000 - val_loss: 94367144.0000
Epoch 9/100
161/161 - 1s - 3ms/step - loss: 93792848.0000 - val_loss: 94252552.0000
Epoch 10/100
161/161 - 1s - 3ms/step - loss: 93678824.0000 - val_loss: 94138512.0000
Epoch 11/100
161/161 - 1s - 3ms/step - loss: 93565080.0000 - val_loss: 94024800.0000
Epoch 12/100
161/161 - 1s - 3ms/step - loss: 93451760.0000 - val_loss: 93910992.0000
